In [ ]:
 
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import json
import six 
import numpy as np 
import math 
import copy 
import re 
import tensorflow as tf 

In [ ]:
# xây dựng lớp bert cấu hình 
class BertConfig(object):
    """Configuration for 'Bert model' """
    # thiết lập phương thức khởi tạo 
    def __init__(self, 
                vocab_size , hidden_size=768, 
                num_hidden_layers =12, num_attention_heads=12, 
                intermediate_size=3072, hidden_act="gelu",
                hidden_dropout_prob=0.1, attention_probs_dropout_prob = 0.1,
                max_position_embeddings = 512, type_vocab_size = 16,
                initializer_range=0.2):
        """Constructs BertConfig. Xây dựng cấu hình cho Bert.
        
        Args : vocab_size : Kích thước của đầu vào mô hình bert 
             hidden_size : Kích thước lớp mã hóa và lớp tổng hợp . 
             num_hidden_layers : Số lượng đầu chú ý cho mỗi lớp attention trong bộ phân mã hóa Transformer 
             intermediate_size : Kích thước trung gian của lớp chuyển tiếp trong bộ mã hóa Transformer
             hiddent_act : Hàm kích hoạt biến đổi đầu vào trong lớp mã hóa và lớp tổng hợp. 
             max_position_embedding : Độ dài tối đa mà mô hình này có thể được sử dụng . 
             type_vocab_size : Kích thước của từ vựng "token_type_ids" đã được chuyển vào trong BertModel.
             initializer_range: Giá trị chuẩn để khởi tạo tất cả các , ma trận trọng số . 
        """
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.num_hidden_layers = num_hidden_layers
        self.num_attention_heads = num_attention_heads
        self.hidden_act = hidden_act
        self.intermediate_size = intermediate_size
        self.hidden_dropout_prob = hidden_dropout_prob
        self.attention_probs_dropout_prob = attention_probs_dropout_prob
        self.max_position_embeddings = max_position_embeddings
        self.type_vocab_size = type_vocab_size
        self.initializer_range = initializer_range

    # xây dựng một phương thức lớp nhận một đối tượng python là một từ điển các tham số cấu hình 
    # và trả về môtj thể hiwnwj của lơp Bert Congfig với các thuộc tính tương ứng với các kháo và giá trị 
    # trong từ điển.
    @ classmethod 
    def from_dict(cls, json_object):
        """Xây dựng a 'BertConfig' từ một từ điển tham số python """
        config = BertConfig(vocab_size=None)
        # sử dụng hàm six.iteritems để duyệt qua các cặp khóa và giá trị trong từ điển 
        for (key , value) in six.iteritems(json_object):
            # gán chúng vào __dict__ của thể hiện cấu hình 
            config.__dict__[key] = value 
        return config 
    
    # Xây dựng thêm 1 phương thức lớp để đọc các file json 
    # và trích suất các đoạn văn bản từ file nguồn 
    @classmethod
    def from_json_file(cls , json_file):
        """Xây dựng một BerConfig từ một tệp tham số json. """
        # tải danh sách file json và cho phép đọc gán vào reader 
        with tf.gfile.GFile(json_file, "r") as reader: 
            # gán biến text = laod reader 
            text = reader.read()
        # sử dụng hàm json.loads để chuyển đổi nó thahf một đối tượng Python là một từ điển
        # sử dụng from_dict để trả về một thể hiện của lớp BertConfig từ từ điển đó 
        return cls.from_dict(json.loads(text))
    
    # xy dựng phương thứ to_dict 
    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        # tạo một bản sao sâu từ một từ điển thể hiện trong bertConfifg mà ta đã tríc xuất từ file json
        output = copy.deepcopy(self.__dict__)
        return output
    # xây dựng phương thức thể hiện, nhưng nó trả về một chuỗi JSON chứa các thuộc tính
    # và giá trị của thể hiện cấu hình. Phương thức này gọi phương thức to_dict để lấy từ điển Python từ thể hiện cấu hình, 
    # và sau đó sử dụng hàm json.dumps để chuyển đổi nó thành một chuỗi JSON. 
    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"



In [ ]:
# Xây dựng mô hình Bert 
class BertModel(object):
    """Bert model ("Bidirectional Encoder Representation from Transformer.)

    
    Chuyển đổi thành id mã thông báo với WordPiece 
    Input_ids = tf.constant([[31, 51, 99], [15, 5, 0]])
    Input_mask = tf.constant([[1, 1 , 1][1, 1, 0]])
    token_type_ids = tf.constant([[0, 0, 1][0, 2, 0]])

    config = modeling.BertConfig(vocab_size= 32000, hidden_size =512 , 
        num_hidden_layers=8 , num_attention_heads=6 , intermediate_size =1024)

    model = modeling.BertModel(config = config , is_training =True , input_ids= input_ids , 
        inpuut_mask = input_mask , token_type_ids = token_type_ids)

    label_embedding = tf.get_variable(..)
    pooled_output = model.get_pooled_output()
    logits = tf.matmul(pooled_output , labels_embeddings)

    """
    # Thiết lập phương thức khởi tạo 
    def __init__(self, config , is_training , input_ids , 
                 input_mask = None , token_type_ids=None , 
                 use_one_hot_embeddings=False, scope=None):
        """Constructor for BertModel.

        Args:
        config: `BertConfig` instance.
        is_training: bool. true for training model, false for eval model. Controls
            whether dropout will be applied.
        input_ids: int32 Tensor of shape [batch_size, seq_length].
        input_mask: (optional) int32 Tensor of shape [batch_size, seq_length].
        token_type_ids: (optional) int32 Tensor of shape [batch_size, seq_length].
        use_one_hot_embeddings: (optional) bool. Whether to use one-hot word
            embeddings or tf.embedding_lookup() for the word embeddings.
        scope: (optional) variable scope. Defaults to "bert".

        Raises:
        ValueError: The config is invalid or one of the input tensor shapes
            is invalid.
        """
        # tạo một biến config = giá trị sao chép của config 
        config = copy.deepcopy(config)
        # kiểm tra xem có đang khoong owr trong trạng thái huấn luyện không 
        if not  is_training: 
            # gán cho biến rời bở cho các lớp trạng thái ẩn  = 0.0
            config.hidden_dropout_prob = 0.0
            # và tham số rời bỏ của lớp attention = 0.0
            config.attemtion_probs_ropout_prob = 0.0

        # truwowngf hopwj conf laij tuwcs đang ở trong trạng thái đào tạo 
        # lấy ra hình dạng đầu vào input_shape  từ input_ids theo 2 chiều 
        input_shape = get_shape_list(input_ids , expected_rak =2)
        # lấy ra kích thước lô và độ dài chuỗi đầu vào từ input_shape
        batch_size =input_shape[0]
        seq_length = input_shape[1]

        # kiểm tả trường hợp có tồn tại mặt nạ tokens không 
        if input_mask is None: 
            # nếu không tồn tại gán input_mask = tensor one shape[batch_size , seq_length]
            input_mask = tf.one(shape=[batch_size , seq_length] , stype=tf.int32)

        # kiểm tra xem danh sách token_type_ids có tồn tại không 
        # token_type_ids là 1 danh sách chứa nhãn cho các câu theo thứ tự bắt đầu từ 0-> n câu 
        if token_type_ids is None: 
            # nếu không tồn tại gán token_type_ids = tensor zeros[batch_size seq_length]
            token_type_ids = tf.zeros(shape =[batch_size , seq_length] , dtype=tf.int32)
        
        # tạo một biến phạm vi ngữ cảnh cho mô hình bert 
        # sử dụng with để đặt tất cả thuộc phạm vi 
        with tf.variable_creator_scope(scope , default_name="bert"):
            # tiếp tục đặt một biến nhúng phạm vi 
            with tf.variable_creator_scope("embeddings"):
                """Thực hiện tra cứu nhúng trên phạm vi từ."""
                # lấy ra kêt quả nhúng và bảng kết quả giá trị nhúng bằng cách truy suất qua 
                # phương thức nhúng tra cứu 
                (self.embedding_output , self.embedding_tabel) = embedding_lookup(
                    input_ids= input_ids , 
                    vocab_size = config.vocab_size, 
                    embedding_size = config.hidden_size , 
                    word_embedding_name = "wword_embeddings", 
                    use_one_hot_embeddings= use_one_hot_embeddings
                )

                # Thêm nhúng vị trí và kiểu nhúng mã thông báo , sau đó bình thường hóa lớp 
                # và biểu diễn rời bỏ 
                self.embedding_output = embedding_postprocessor(
                    input_tensor=self.embedding_output,
                    use_token_type=True,
                    token_type_ids=token_type_ids,
                    token_type_vocab_size=config.type_vocab_size,
                    token_type_embedding_name="token_type_embeddings",
                    use_position_embeddings=True,
                    position_embedding_name="position_embeddings",
                    initializer_range=config.initializer_range,
                    max_position_embeddings=config.max_position_embeddings,
                    dropout_prob=config.hidden_dropout_prob)

            # Tạo một biến phạm vi cho bộ phận mã hóa của Bert 
            with tf.variable_creator_scope("encoder"):
                # điều này chuyển đổi một mặt ạn 2D  với hình dạng shape [batch_size , seq_length]
                # thành một mặt nạ 3D với hình dạng shape [batch_size , seq_length , seq_length] 
                # để khi sử dụng cho lớp attention (điểm chú ý của lớp attention)
                attention_mask = create_attention_mask_from_input_mask(
                    input_ids, input_mask)
                
                # chạy máy biến áp xếp trồng chuỗi đầu ra seq_length shape = [batch_size , seq_length , hidded_size]
                self.all_encoder_layers = tranformer_model(
                    input_tensor=self.embedding_output,
                    attention_mask=attention_mask,
                    hidden_size=config.hidden_size,
                    num_hidden_layers=config.num_hidden_layers,
                    num_attention_heads=config.num_attention_heads,
                    intermediate_size=config.intermediate_size,
                    intermediate_act_fn=get_activation(config.hidden_act),
                    hidden_dropout_prob=config.hidden_dropout_prob,
                    attention_probs_dropout_prob=config.attention_probs_dropout_prob,
                    initializer_range=config.initializer_range,
                    do_return_all_layers=True

                )

                # lấy ra kích thước theo chiều cuối cùng của all_encoder_layers 
                # và gán kích thước này vào biến sequence_output
                self.sequence_output = self.all_encoder_layers[-1]
                # chuyển đổi các lớp gộp mã hóa chuỗi 1 tensor shape = [batch_size , seq_length, hidden_size]
                # thành 1 tensor shape [batch_size , hidden_size]. Điều này cần thiết cho cấp bận phân đoạn 
                # hay hoặc cấp độ cặp phân khúc các nhiệm vụ phân loại trong đó cần biểu diễn chiều cố định 
                # của phân loại 
                # tạo một biến phạm vi ngữ cảnh cho các lớp gộp 
                with tf.variable_creator_scope('pooler'):
                    # gộp mô hình bằng cách lấy trạng thái ẩn tương ứng cho token đầu tiên . 
                    # cho rằng điều này đã được đào tạo trước . 
                    # self.sequence_output[:, 0:1, :]: Đoạn mã này sử dụng cắt tỉa trên các chiều của
                    #  self.sequence_output. Trong trường hợp này, nó giữ lại tất cả các chiều từ 0 đến 1 
                    # trên chiều thứ hai (axis 1), và toàn bộ các giá trị trên các chiều khác (chiều thứ nhất và chiều thứ ba). 
                    # Do đó, đầu ra của nó sẽ có kích thước là [batch_size, 1, hidden_size].
                    first_token_tensor = tf.squeeze(self.sequence_output[:, 0:1, :] , axis=1)
                    # ánh xạ đầu ra pooler layer đầu ra của lớp gộp qua một mạng dense 
                    self.pooled_output = tf.keras.layers.Dense(
                        first_token_tensor,
                        config.hidden_size,
                        activation=tf.tanh,
                        kernel_initializer=create_initializer(config.initializer_range)
                    )

    # xây dựng phương thức trả về kết quả của lớp ẩn 
    def get_pooled_output(self):
        return self.pooled_output
    
    # tương tự xây dựng một phương thức trả về đầu ra tuần tự là trạng thái ẩn của khối encoder 
    def get_sequence_ouput(self):
        """Nhận lớp ẩn cuối cùng của bộ phận encoder
        
        Return : 
            Một tensor type = float có dạng [batch_size , seq_length,  hidden_size] tương ứng 
            với lớp ẩn cuối cùng của khối encoder Transformer. 
        """

        return self.sequence_output 
    
    # xây dựng một phương thức trả về một loạt các lớp encoder trong của bert 
    def get_all_encoder_layers(self):
        return self.all_encoder_layers
    
    # xây dựng phương thức trả về kết quả của lớp nhúng 
    def get_embedding_output(self):
        """Gets output of the embedding lookup (i.e., input to the transformer).
            Nhận đầu ra của tra cứu nhúng . đây là đầu vào của Transformer Encoder 
        Returns:
            float Tensor of shape [batch_size, seq_length, hidden_size] corresponding
            to the output of the embedding layer, after summing the word
            embeddings with the positional embeddings and the token type embeddings,
            then performing layer normalization. This is the input to the transformer.
        """
        return self.embedding_output

    # xây dựng phươnh thức trả về bảng tra cứu nhúng văn bản 
    def get_embedding_table(self):
        return self.embedding_table


def gelu(x):
    """Gaussian Error Linear Unit.

    This is a smoother version of the RELU.
    Original paper: https://arxiv.org/abs/1606.08415
    Args:
        x: float Tensor to perform activation.

    Returns:
        `x` with the GELU activation applied.
    """
    cdf = 0.5 * (1.0 + tf.tanh(
      (np.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3)))))
    return x * cdf



def get_activation(activation_string):
    """Maps a string to a Python function, e.g., "relu" => `tf.nn.relu`.

    Args:
        activation_string: String name of the activation function.

    Returns:
        A Python function corresponding to the activation function. If
        `activation_string` is None, empty, or "linear", this will return None.
        If `activation_string` is not a string, it will return `activation_string`.

    Raises:
        ValueError: The `activation_string` does not correspond to a known
        activation.
    """

    # We assume that anything that"s not a string is already an activation
    # function, so we just return it.
    if not isinstance(activation_string, six.string_types):
        return activation_string

    # 
    if not activation_string:
        return None 

    act = activation_string.lower()
    if act == "linear":
        return None
    elif act == "relu":
        return tf.nn.relu
    elif act == "gelu":
        return gelu
    elif act == "tanh":
        return tf.tanh
    else:
        raise ValueError("Unsupported activation: %s" % act)

In [ ]:
# Xây dựng phương thức có chức năng lấy một phần bản đồ gán giữa các biến hiện tại
# và các biến trong một checkpoint 
def get_assigment_map_from_checkpoint(tvars, init_checkpoint):
    """Compute the union of the current variables checkpoint variables.
        Tính toán sự khác biệt của các biến hiện tại và các biến điểm kiểm tra 
    
    """
    # khưởi tạo 2 từ điển assigment dẽ chứa các cặp key và value là tên của biến đã được khởi tạo 
    # từ checkpoint . 
    assignment_map = {}
    # và từ điển init này sẽ chứa các khóa là tên của các biến trong mô hình hiện tại và các giá 
    # trị là đối tượng biến tương ứng . 
    initialized_variable_names = {}

    # tạo một từ điển có thứ tự có các mục được sắp xếp theo thứ tự chúng được thêm vào 
    name_to_variable = collections.OrderedDict()
    # duyệt qua dạn sách tvar là một danh sách các biến huấn luyện trong mô hình hiện tại , 
    # được truyền vào hàm như một tham số đầu vào 
    for var in tvars: 
        # gán biến name = var.name tức là thuộc tính name của biến var 
        name = var.name 
        # gán biến m bằng kết quả của hàm re.match với tham số đầu vào là một biểu thức chính quy 
        # ((.*):\d+) . có nghĩa là bắt đầu bằng bất kỳ thứ tự nào , theo sau là dấu 2 chấm , rồi kết thúc 
        # băng một hiawcj nhiều chữ số vd : "weight:0", nếu name khớp trả về 1 đối tượng match 
        # còn lại trả về None 4
        m = re.match("^(.*):\\d+$", name)
        # kiểm tra xem m có tồn tại không 
        if m is not None:
            # nếu có ta nhóm m lại thành nhóm thứ nhất 
            name = m.group(1)
        # thêm các giá trị name và giá trị tương ứng vào từ điển theo thứ tự 
        name_to_variable[name] = var

    # tạo một danh sách các biến từ init_checkpoint là một chuỗi chỉ định đường dẫn đến tập checkpoint 
    # được truyền vào hàm như một tham số đầu vào . Hàm list_variable dùng để lấy một tập tên và kích 
    # thước của các biến trong cjeckpoint
    init_vars = tf.train.list_variables(init_checkpoint)

    # tạo một từ điển mà các mục được sắp xếp theo thứ tự mà chúng được thêm vào 
    assignment_map = collections.OrderedDict()
    # duyệt qua danh sách init_vars
    for x in init_vars:
        # tríc xuất ra tên và giá trị của các biến tương ứng 
        (name , var) = (x[0], x[1])
        # kiểm tra xem tên biến có tồn tại trong từ điển name_to_variable không 
        if name not in name_to_variable: 
            # nếu không tồn tại thì tiếp tục 
            continue 
        # trường hợp tức là có tồn tại 
        # tạo một từ điển map chứa tên của các biến được huân luyện trong mô hình 
        assignment_map[name] = name
        # Thêm hai mục vào initialized_variable_names, có khóa là name và name + “:0”,
        # và giá trị đều là 1. 
        # Điều này có nghĩa là đánh dấu biến name đã được khởi tạo từ checkpoint.
        initialized_variable_names[name] = 1
        initialized_variable_names[name + ":0"] = 1

    return (assignment_map, initialized_variable_names)


In [ ]:
# xây dựng phương thức rời bỏ dropout 
def dropout(input_tensor, dropout_prob):
    """Perform dropout.

    Args:
        input_tensor: float Tensor.
        dropout_prob: Python float. The probability of dropping out a value (NOT of
        *keeping* a dimension as in `tf.nn.dropout`).

    Returns:
        A version of `input_tensor` with dropout applied.
    """
    # kiểm tra xem có tồn tại tỷ lệ dropout hoặc tywr lệ này = 0
    if dropout_prob is None or dropout_prob == 0.0:
        # thì trả về nguyên kết quả là tensor đầu vào lớp này 
        return input_tensor
    # trường hợp có tồn tại tỷ lệ bỏ học 
    output = tf.nn.dropout(input_tensor, 1.0 - dropout_prob)
    # ta trả về kết quả cho đầu vào được biến đổi 
    return output

# Xây phương thức khởi tạo cho lớp layernormalization 
def layer_norm(input_tensor, name=None):
    """Run layer normalization on the last dimension of the tensor"""
    # trả về kết quả của tensor sau khi chuyển qua lớp layernorm
    return tf.contrib.layers.layer_norm(
        inputs=input_tensor, begin_norm_axis=-1, begin_params_axis=-1, scope=name
    )
# xây dựng một phương thức kết hợp của lớp drôput và layernorm 
def layer_norm_and_dropout(input_tensor, dropout_prob, name=None):
    """Runs layer normalization followed by dropout."""
    # add layernorma
    output_tensor = layer_norm(input_tensor, name)
    # add dropout layer
    output_tensor = dropout(output_tensor, dropout_prob)
    return output_tensor

# xây dựng một phương thức khởi tạo các trọng số cho mô hình 
def create_initializer(initializer_range=0.02):
    """Creates a `truncated_normal_initializer` with the given range."""
    # khởi tạo các trọng số với tỷ lệ được cắt bớt bởi độ lệch chuẩn . 
    return tf.truncated_normal_initializer(stddev=initializer_range)


# Xây dựng một lớp tra cứu nhúng 
def embedding_lookup(input_ids , vocab_size, embedding_size=128 , 
                     initializer_range= 0.02 , word_embedding_name ="word_embeddings",
                     use_one_hot_embeddings=False
                     ):
    """Looks up words embeddings for id tensor.
        Tra cứu các từ nhúng cho id tensor . 

    Args: 
        input_size: int . Size of the embedding vocabulary . 
        vocab_size: int. Size of the embedding vocabulary.
        embedding_size: int. Width of the word embeddings.
        initializer_range: float. Embedding initialization range.
        word_embedding_name: string. Name of the embedding table.
        use_one_hot_embeddings: bool. If True, use one-hot method for word

        embeddings. If False, use `tf.gather()` vocab_size: int. Size of the embedding vocabulary.
        embedding_size: int. Width of the word embeddings.
        initializer_range: float. Embedding initialization range.
        word_embedding_name: string. Name of the embedding table.
        use_one_hot_embeddings: bool. If True, use one-hot method for word
        embeddings. If False, use `tf.gather()`

    Returns: 
        float Tensor of shape [batch_size , seq_length , embedding_size].
    """
    # This function assumes that the input is of shape [batch_size, seq_length,
    # num_inputs].
    # If the input is a 2D tensor of shape [batch_size, seq_length], we
    # reshape to [batch_size, seq_length, 1].
    
    # kiểm tra xem hình dạng của tensor input_ids hiện tại có số chiều n_dims == 2 hay không 
    if input_ids.shape.ndims == 2:
        # thêm vào tensor input_ids 1 chiều shape = 1 vào vị trí cuối cùng của tensor 
        # kết quả ta nhận được 1 tensor input_ids shape = [batch_size , seq_length , 1]
        input_ids = tf.expand_dims(input_ids , axis=[-1])
    
    # gán biến embedding cho kết quả của hàm tf.get_variable hàm này trả về một kết quả là một
    # bảng nhúng từ một bộp từ vựng có kích thước vocab_size từ kích thước nhúng = embedding_size 
    # 
    embedding_table = tf.get_variable(
        name = word_embedding_name , 
        shape = [vocab_size , embedding_size],
        initializer = create_initializer(initializer_range)
    )

    # tạo một ma trận flat_input có shape = [batch_size , seq_length], là chiều cuối cùng của tensor input_ids
    # với giá trị - 1 nghĩa là chiều còn lại sẽ được tính tự động dựa vào số chiều còn lại 
    flat_input_ids = tf.reshape(input_ids , [-1])
    # kiểm tra xem use_one_hot_embeddings có tồn tại không 
    if use_one_hot_embeddings: 
        # tạo một biến là use_one_hot_ids = 1 ma trận one_hot shape = [batch_size , sqg_length, vocab_size ]
        one_hot_input_ids = tf.one_hot(flat_input_ids , depth=vocab_size)
        # Tạo một tensor ouput bằng kết quả của phép nhân tensor on_hot_input_ids , embedding_table 
        # shape [batch_size ,seq_length, vocab_size] * [vocab_size , embeddinh_size]
        # trả về shape = [batch_size ,seq_length, embeding_size]
        output = tf.matmul(one_hot_input_ids , embedding_table)

    # trường hợp còn lại tức không tồn tại one_hot_embeddings 
    else: 
        # sử dụng hàm tập trung tf.gather để kết hợp 2 tensor embedding_tabel và flat_input
        # shape = [vocab_size , embedding_size] và [batch_size * seq_length]
        # giá trị mới tensor kết quả là các phần tử của params được lấy theo các chỉ số của indices
        # shape = [batch_size ,seq_length, embeding_size] 
        # ở đây ta đặt mặc định axis = 0 tức là sẽ bỏ đi chiều đầu của tensor prams và lấy 
        # kích thước theo ma trận indices 
        output = tf.gather(prams=embedding_table , indices=flat_input_ids)  

        # sử dụng phương thức get_shape_list để lấy ra tất acr hình dnagj của 1 tensor dưới dạng số nguyên 
        input_shape = get_shape_list (input_shape)

        # định hình lại kích thước đầu ra với hình dnagj input_shape 
        # = [batch_size, seq_length ,num_inputs].
        # và shape = [batch_size ,seq_length, embeding_size] 
        output = tf.reshape(output , input_shape[0:, -1] # lấy 2 kích thước đầu 
                            # kết quả sẽ được 1 tensor shape  = (batch_size, seq_length, num_inputs * embedding_size)
                            + [input_shape[-1] * embedding_size])
        
        # trả về kết quả và bản tra cứu nhúng 
        return (output, embedding_table)
    

# xây dựng phương thức embedding_posprocessor (phương thức xử lý nhúng hậu kỳ)
# Thực hiện các bước xử lý sau cùng trên một tensor nhúng từ 
# trả về 1 tensor có cùng kích thước với tensor đầu vào 


def embedding_postprocessor(input_tensor,
                            use_token_type=False,
                            token_type_ids=None,
                            token_type_vocab_size=16,
                            token_type_embedding_name="token_type_embeddings",
                            use_position_embeddings=True,
                            position_embedding_name="position_embeddings",
                            initializer_range=0.02,
                            max_position_embeddings=512,
                            dropout_prob=0.1):
    """Thực hiện quá trình sử lý hậu kỳ trên một tensor nhúng từ. 
       Performs various post-processor on a word embedding tensor. 

    Args: 
        input_tensor: float Tensor of shape [batch_size, seq_length,
            embedding_size].
        use_token_type: bool. Whether to add embeddings for `token_type_ids`.
        token_type_ids: (optional) int32 Tensor of shape [batch_size, seq_length].
            Must be specified if `use_token_type` is True.

        token_type_vocab_size: int. The vocabulary size of `token_type_ids`.
        token_type_embedding_name: string. The name of the embedding table variable
            for token type ids.

        use_position_embeddings: bool. Whether to add position embeddings for the
            position of each token in the sequence.
        position_embedding_name: string. The name of the embedding table variable
            for positional embeddings.

        initializer_range: float. Range of the weight initialization.
        max_position_embeddings: int. Maximum sequence length that might ever be
            used with this model. This can be longer than the sequence length of
        input_tensor, but cannot be shorter.
        dropout_prob: float. Dropout probability applied to the final output tensor.

    Returns:
        float tensor with same shape as `input_tensor`.

    Raises: 
        ValueError : One of the tensor shapes or input values is invalid.
    """  
    # lấy ra hình dạng của input_tensor với số chiều mong đợi = 3
    input_shape = get_shape_list(input_tensor, expected_rank=3)
    # gán 3 biến batch_size , seq_length , width với giá trị là 
    # kích thước của 3 chiều tương ứng 
    batch_size = input_shape[0]
    seq_length = input_shape[1]
    width = input_shape[2]

    # gán tensor đâu ra = tensor đâu vào 
    output = input_tensor

    # kiểm tra xem biến bool use_token_type là biến cho biết có thêm phần nhúng cho token_type_id 
    # hay không
    if use_token_type: 
        # đồng thời kiểm tra xem token_type_id tensor này có tồn tại không 
        if token_type_ids is None: # nếu không 
            # trả về một cảnh báo 
            raise ValueError ("`token_type_ids` must be specified if"
                       "`use_token_type` is True.")
    
    # tạo một biến token_type_table có chức năng dùng để tra cứu bộ nhúng cho token_type_id
    # kết quả trả về  tensor có chứa các vector nhúng cho các loại token khác nhau 
    # kết qyar này sẽ được sử dụng để thêm vào tensor nhúng từ cho loại tokens của mỗi token trong 
    # chuỗi đầu vào 
    token_type_table = tf.get_variable(
        name = token_type_embedding_name, 
        shape =[token_type_vocab_size, width],
        initializer = create_initializer(initializer_range)
    )

    # từ điển điển này sẽ nhỏ nên cần thực hiện one-hot ở đây vì nó luôn nhanh hơn đối 
    # với từ vựng nhỏ 
    # tạo một tensor flat_token có hình dạng bằng với tensor token_type_ids 
    # với tham số - được hiểu là chiều cuối cùng sẽ được suy ra dựa vào 2 chiều còn lại 
    flat_token_type_ids = tf.reshape(token_type_ids, [-1])
    # tạo một tensor one_hot shape = [batch_size , seq_length, token_type_vocab_size ]
    one_hot_ids = tf.one_hot(flat_token_type_ids, dept=token_type_vocab_size)
    
    # nhân 2 ma trận là one_hot_ids và token_type_table với phép nhân hàng và cột matmul 
    # [batch_size , seq_length, token_type_vocab_size ] * [token_type_vocab_size, width]
    # kết quả là 1 tensor mới shape = [batch_size * seq_length , width]
    token_type_embeddings = tf.matmul(one_hot_ids, token_type_table)

    # định hình lại tensor này với hình dạng batch_Size , seq_length , width 
    token_type_embeddings = tf.reshape(token_type_embeddings,
                                       [batch_size, seq_length, width])
    # sau đó cộng ma trận đầu ra với am trận này 
    output += token_type_embeddings

    # kiểm tra xem use_position_embedding có tồn tại không 
    # đây là ma trận nhúng vị trí các tokens 
    if use_position_embeddings: 
        # tạo một phép kiểm tra để đảm bảo rằng seq_length của 1 tensor đầu vào không vượt quá 
        # max_position_len chỉ số tối đa của 1 đoạn đầu vào được đánh dấu 
        assert_op = tf.assert_less_equal(seq_length, max_position_embeddings)
        # tạo một ngữ cảnh quản lý để đảm bảo răng các hoạt động trong khối này chỉ được 
        # thực hiện sau khi assert_op được thực hiện 
        with tf.control_dependencies([assert_op]):
            # tạo một biên mới full_position nhận kết quả chứa danh sách các vector nhúng cho các
            # tensor đầu vào 
            full_position_embeddings = tf.get_variable(
                name=position_embedding_name,
                shape=[max_position_embeddings, width],
                initializer=create_initializer(initializer_range))

            # khởi tạo mọt tensor  nhúng bắt đầu  0-> seq_length - 1 với hàm slice
            position_embeddings = tf.slice(full_position_embeddings, [0, 0],
                                     [seq_length, -1])
            # Gán giá trị cho biến num_dims bằng cách lấy độ dài của danh sách chứa kích thước của tensor output
            num_dims = len(output.shape.as_list())

            # tạo một danh sách position_broadcast để chứa kích thước thước của ma trận 
            # position embeddings  danh sách này có số phần tử bằng num_dims là số chiều 
            # của ma trận position_embedding
            position_broadcast_shape = []
            # lặp 1 lần từ 0 -> số num_dim -1 
            for _ in range(num_dims - 2):
                # gán cho các phần tử đầu tiên của danh sách = 1
                position_broadcast_shape.append(1)
            # còn 2 phần  tử cuối cùng của dnah sách được gán bằng  seq_length và width 
            position_broadcast_shape.extend([seq_length, width])
            # định hình lại ma trận position_emnbeddings theo kích thước của ma trận position_boardcast
            # mục đích là để có thể cộng được nó với ma trận output 
            position_embeddings = tf.reshape(position_embeddings,
                                            position_broadcast_shape)
            # công ma trận output với ma trận position_embedding 
            # kết quả là tensor shape = [batch_size , seq_lengh ,width]
            output += position_embeddings
    
    # áp dụng phương thức chuẩn hóa cho lớp đầu ra 
    output = layer_norm_and_dropout(output, dropout_prob)
    # trả về kết quả sau khi được chuẩn hóa 
    return output

In [ ]:
# Xây dưng một phương thức tạp mặt nạ chú ý từ đầu vào 
def create_attention_mask_from_input_mask(from_tensor , to_mask):
    """Create 3d Attention mask from 2D tensor mask. 
        Tạo ra một mặt nạ chú ý 3 chiều từ tensor mask 2 chiều. 

    Args: 
        from_tensor : 2D or 3D Tensor of shape [batch_size , from_seq_length,...].
        to_mask : Int32 Tensor of shape [batch_size, to_seq_length].


    Returns: 
        Float tensor of shape [batch_size, from_seq_length , to_seq_length].
    """
    # tạo một biến from_shape = hình dạng của tensor đầu vào from_tensor là một ma trận có 2
    # hoặc 3 chiều ta sẽ lấy toàn bộ số chiều của nó
    from_shape = get_shape_list(from_tensor, expected_rank=[2 ,3])
    # gán batch_size  = kích thước from_shape theo chiều 0
    batch_size = from_shape[0]
    # tương tự lấy ra seq_length 
    from_seq_length = from_shape[1]

    # lấy ra hình dạng của tensor mask là 1 tensor 2 chiều 
    to_shape = get_shape_list(to_mask , expected_rank=2)
    # gán cho biến to_seq_length = chiều thứ 2 được lấy bởi to_shape
    to_seq_length = to_shape[1]

    # tạo một tensor là to_mask dtype = float 32
    # shape = [batch_size , seq_length] mục đích để khi nhân tensor này với 
    # tensor boardcasr_one để có được tensor đầu ra mong muồn 
    to_mask = tf.cast(
        tf.reshape(to_mask , [batch_size, 1 , to_seq_length] , dtype=tf.float32)
    )

    # ta tạo một tensor boadcast_one tươ ng tụ 
    broadcast_ones = tf.ones(
      shape=[batch_size, from_seq_length, 1], dtype=tf.float32)

    # Here we broadcast along two dimensions to create the mask.
    mask = broadcast_ones * to_mask

    return mask


# xây dựng lớp chú ý cho khối transformer 

def attention_layer(
        from_tensor ,to_tensor, attention_mask=None , num_attention_heads = 1, 
        size_per_head = 512 , query_act =None , key_act =None , value_act =None ,
        attention_probs_dropout_prob = 0.0, initializer_range=0.02, 
        do_return_2d_tensor =False , batch_size =None , from_seq_length=None ,
        to_seq_length=None
    ):

    """Performs multi-headed attention from `from_tensor` to `to_tensor`.

        This is an implementation of multi-headed attention based on "Attention
        is all you Need". If `from_tensor` and `to_tensor` are the same, then
        this is self-attention. Each timestep in `from_tensor` attends to the
        corresponding sequence in `to_tensor`, and returns a fixed-with vector.

        This function first projects `from_tensor` into a "query" tensor and
        `to_tensor` into "key" and "value" tensors. These are (effectively) a list
        of tensors of length `num_attention_heads`, where each tensor is of shape
        [batch_size, seq_length, size_per_head].

        Then, the query and key tensors are dot-producted and scaled. These are
        softmaxed to obtain attention probabilities. The value tensors are then
        interpolated by these probabilities, then concatenated back to a single
        tensor and returned.

        In practice, the multi-headed attention are done with transposes and
        reshapes rather than actual separate tensors.

    Args: 
        from_tensor : float multi_headed attention of shape [batch_size , sqe_length , from_width]
        to_tensor: Float tensor of shape [batch_size , to_seq_length , to_width].
        attention_mask: (không bắt buộc) : int32 Tensor of shape [batch_size , seq_length , to_seq_length].
            The values should 0 or 1. The attention scores will effectively be set to -infinity for any 
            positions in the mask tha are 0 , and will be unchangeed for positions that are 1. 
        num_attention_heads : int . Number of attention heads . 
        size_per_head : Int . Size of each attention head. 
        query_act: (optional) Activation function for the query transform.
        key_act: (optional) Activation function for the key transform.
        value_act: (optional) Activation function for the value transform.
        attention_probs_dropout_prob: (optional) float. Dropout probability of the
            attention probabilities.

        initializer_range: float. Range of the weight initializer. 
        do_return 2d_tensor: bool .If True , the output will be of shape [batch_size * from_seq_length,
            num_attention_heads * size_per_head]. If false, the output will be of shape [batch_size , from_seq_length,
            num_attention_heads * size_per_head].
        
        
        batch_size: (Optional) int. If the input is 2D, this might be the batch size
            of the 3D version of the `from_tensor` and `to_tensor`.
        from_seq_length: (Optional) If the input is 2D, this might be the seq length
            of the 3D version of the `from_tensor`.
        to_seq_length: (Optional) If the input is 2D, this might be the seq length
            of the 3D version of the `to_tensor`.
    
    Retuns : 
        Float Tensor of shape [batch_size , from_seq_length, num_attentiom_heads * size_per_head].
            If do return 2d Tensor is True , this will be of shape [batch_size * from_seq_length, 
             num_attention_heads * size_per_head]

     
    Raises:
        ValueError: Any of the arguments or tensor shapes are invalid.   
    """

    # Xaay duwngj phuwowng thuwcs chuyển đổi hình dnagj cho ma trận score
    # là các ma trận q k , v 
    def transpose_for_scores(input_tensor , batch_size, num_attention_heads, 
                             seq_length , width):
        # ddinhj hinhf lại input tensor shape = [batch_size , seq_length , num_attention, width]
        output_tensor = tf.reshape(
             input_tensor, [batch_size, seq_length, num_attention_heads, width])

        # Chuyển vị lại  output_tensor shape = [batch_size ,num_attention , seq_length , width]
        output_tensor = tf.transpose(output_tensor, [0, 2, 1, 3])
        # trả về tensor đã được chuyển vị 
        return output_tensor
    
    # lấy ra hình dnahj cuảt 2 tensor là from_tensor và to_tensor với kích thước mong đợi
    # gán cho nó 2 biến ;à from_shape  và to_shape 
    from_shape = get_shape_list(from_tensor, expected_rank=[2, 3])
    to_shape = get_shape_list(to_tensor, expected_rank=[2,3])

    # kiểm tra xem số chiều của to_shape và from_shape có bằng nhau không
    if len(from_shape) != len(to_shape):
        # trả về một cảnh báo lỗi 
        raise ValueError(
        "The rank of `from_tensor` must match the rank of `to_tensor`.")
    
    # trường hợp nếu số chiều được lấy from_shape = 3
    if len(from_shape) == 3:
        # lấy ra 2 kích thước 0 , và và gá cho 3 biến là batcj_size , seq_length ,to_seq_length
        batch_size = from_shape[0]
        from_seq_length = from_shape[1]
        to_seq_length = from_shape[1]

    # trường hợp < 3 thì ta kiểm tra dàng buộc và đưa ra một cảnh báo lỗi 
    elif len(from_shape) == 2:
        if (batch_size is None or from_seq_length is None or to_seq_length is None):
            raise ValueError(
                "When passing in rank 2 tensors to attention_layer, the values "
                "for `batch_size`, `from_seq_length`, and `to_seq_length` "
                "must all be specified.")

     # Scalar dimensions referenced here:
    #   B = batch size (number of sequences)
    #   F = `from_tensor` sequence length
    #   T = `to_tensor` sequence length
    #   N = `num_attention_heads`
    #   H = `size_per_head`

    # reshape 2 tensor from_tensor và to_tensor thành ma trận 2 chiều 
    from_tensor_2d = reshape_to_matrix(from_tensor)
    to_tensor_2d = reshape_to_matrix(to_tensor)

    # `query_layer` = [B*F, N*H] shape = [batch_size, seq_length , num_heads , head_dim(size_per_head)]
    query_layer = tf.layers.dense(
        from_tensor_2d,
        num_attention_heads * size_per_head,
        activation=query_act,
        name="query",
        kernel_initializer=create_initializer(initializer_range))

    # `key_layer` = [B*T, N*H] shape = [batch_size, seq_length , num_heads , head_dim(size_per_head)]
    key_layer = tf.layers.dense(
        to_tensor_2d,
        num_attention_heads * size_per_head,
        activation=key_act,
        name="key",
        kernel_initializer=create_initializer(initializer_range))

    # `value_layer` = [B*T, N*H] shape = [batch_size, seq_length , num_heads , head_dim(size_per_head)]
    value_layer = tf.layers.dense(
        to_tensor_2d,
        num_attention_heads * size_per_head,
        activation=value_act,
        name="value",
        kernel_initializer=create_initializer(initializer_range))

    # `query_layer` = [B, N, F, H] chuyển vị tensor q thành tensor có dạng [batch_size ,num_heads , seq_length, nem_per_h ]
    # để có thể nhân tensor q với tensor k kết quả nhận được 1 tensor score
    query_layer = transpose_for_scores(query_layer, batch_size,
                                        num_attention_heads, from_seq_length,
                                        size_per_head)

    # `key_layer` = [B, N, T, H] chuyển vị tensor q thành tensor có dạng [batch_size ,num_heads , seq_length, nem_per_h ] 
    key_layer = transpose_for_scores(key_layer, batch_size, num_attention_heads,
                                    to_seq_length, size_per_head)

    # Tính tích số q . K.T để có điuwocj điểm chú ý 
    # âttantion_score shape = [ batch_size , num_heads , seq_lengt , num_per_head]
    attention_scores = tf.matmul(query_layer, key_layer, transpose_b=True)
    # thực hiện phép tính chia điểm score  cho căn bậc 2 dk tức là số chiều biểu diễn nhúng 
    # kết quả ở đây là 25 
    attention_scores = tf.multiply(attention_scores,
                                 1.0 / math.sqrt(float(size_per_head)))
    
    # kiểm tra xem có tồn tại tensor mặt nạ tự chú ý 
    # attention_mask là 1 tensor int32  shape [batch_size, from_seq_length , to_seq_length]
    if attention_mask is not None: 
        # Thêm 1 chiều có size = 1 vào  vị trí thứ 2 của tensor 
        # kết quả là một tensor shape = [batch_size ,1 , num_heads , seq_length]
        attention_mask = tf.expand_dims(attention_mask , axis=[1])
        # mặt nạ chú ý mang các giá trị 1.0 cho các vị trí được giữ lại và 0.0 
        # cho vị trí các tokens bị che đi . Thao tác này sẽ tạo ra một tensor
        # 0.0 cho các vị trí được giữ lại và -10000 cho các vị trí được che đi 
        adder = (1.0 -tf.cast(attention_mask, tf.float32)) *- 10000.0

        # sau đó cộng ma trận scores là kết quả của phép nhân ma trận q.k 
        # với mặt nạ chú ý để che đi những chỉ số có điểm score quá nhỏ 
        attention_scores += adder 

    # Bình thường hóa attention_scores bằng xác suất với một hàm softmax
    attention_probs = dropout(attention_scores)

    # Thêm một lớp dropout cho attention 
    attention_probs = dropout(attention_probs ,attention_probs_dropout_prob )

    # định hình lại vector v với kích thước [batch_size , seq_length , num-heads , num_per_head]
    value_layer = tf.reshape(
        value_layer , [batch_size , to_seq_length , num_attention_heads , size_per_head]
    )
    # chuyển vị vector V này về hình dạng [batch_size , num_heads , seq_length , num_per_head]
    # để có thể thực hiện nhân tensor Attention_probs với vector v như đúng công thức định nghĩa 
    value_layer = tf.transpose(
        value_layer , [0,2,1,3]
    )

    # Tạo một vector ngữ cản context_layer là kết quả của phép nhân tensor v với kết quả của Q.K.T /^ DK 
    # kết quả là một tensor mới shape = [ batch_size , num_heads , seq_length, num_per_head]
    context_layer = tf.matmul(attention_probs, value_layer)    

    # chuyển vị tensor ngữ cảnh về hình dạng ban đầu 
    # shape = [batch_size , seq_length , num_head , num_per_head]
    context_layer = tf.transpose(context_layer, [0, 2, 1, 3])

    # kiểm tra xem biến do_return _tensor có = True không 
    # do return tensor là một biến boolen 
    if do_return_2d_tensor:
    # `context_layer` = [B*F, N*H]
        context_layer = tf.reshape(
            context_layer,
            [batch_size * from_seq_length, num_attention_heads * size_per_head])
    # trường hợp còn lại tức do_retunr  = false 
    else:
        # `context_layer` = [B, F, N*H]
        context_layer = tf.reshape(
            context_layer,
            [batch_size, from_seq_length, num_attention_heads * size_per_head])

    # Trả về Tensor context 
    return context_layer



# Xây dựng phương thức khởi tạo cho một mô hình Transformer 
def transformer_model(input_tensor, attention_mask=None, 
                      hidden_size = 768 , num_hidden_layers= 12, 
                      num_attention_heads =12, intermediate_size = 3072,
                      intermediate_act_fn=gelu,
                      hidden_dropout_prob=0.1,
                      attention_probs_dropout_prob=0.1,
                      initializer_range=0.02,
                      do_return_all_layers=False            
        ):
    
    """Multi-headed, multi-layer Transformer from "Attention is All You Need".

        This is almost an exact implementation of the original Transformer encoder.

        See the original paper:
        https://arxiv.org/abs/1706.03762

        Also see:
        https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/models/transformer.py

    Args:

        Input_tensor : Float Tensor of shape [batch_size , seq_length , hidden_size]
        attention_mask: (optional) int32 Tensor of shape [batch_size, seq_length,
            seq_length], with 1 for positions that can be attended to and 0 in
            positions that should not be.
        hidden_size: int. Hidden size of the Transformer.
        num_hidden_layers: int. Number of layers (blocks) in the Transformer.
        num_attention_heads: int. Number of attention heads in the Transformer.
        intermediate_size: int. The size of the "intermediate" (a.k.a., feed
            forward) layer.
        intermediate_act_fn: function. The non-linear activation function to apply
            to the output of the intermediate/feed-forward layer.
        hidden_dropout_prob: float. Dropout probability for the hidden layers.
        attention_probs_dropout_prob: float. Dropout probability of the attention
            probabilities.
        initializer_range: float. Range of the initializer (stddev of truncated
            normal).
        do_return_all_layers: Whether to also return all layers or just the final
            layer.

    Returns:
        float Tensor of shape [batch_size, seq_length, hidden_size], the final
        hidden layer of the Transformer.

    Raises:
        ValueError: A Tensor shape or parameter is invalid.
    """
    # kiểm tra xem hidden_size có chia hết cho num_attention_heads 
    if hidden_size % num_attention_heads != 0:
        # nếu không chia hết đưa ra cảnh báo 
        raise ValueError(
            "The hidden size (%d) is not a multiple of the number of attention "
            "heads (%d)" % (hidden_size, num_attention_heads))
    
    # khởi tạo một biến là attention_head_size biểu thị cho kích thước của mỗi đầu chú ý 
    attention_head_size = int(hidden_size / num_attention_heads)
    # sử dụng hàm get_shape_list để lấy ra kích thước của tensor input với kích thước mong đợi 
    # là 3 chiều 
    input_shape = get_shape_list(input_tensor , expected_rank=3)
    # gán các chiều lần lượt được lấy bởi input_shape 
    # lần lượt cho các biến batch_size  , seq_length , input_width . 
    batch_size = input_shape[0]
    seq_length = input_shape[1]
    input_width = input_shape[2]

    # Transformer thực hiện tổng phần dư trên tất cả các lớp đầu vào nên cần có cùng kích thước với 
    # hidden_size 
    # kiểm tra xem đầu vào input_width có cùng kích thước với hidden_size 
    if input_width != hidden_size: 
        # nếu không có cùng kích thước lập tức đưa ra cnahr báo 
        raise ValueError ("The width of input tensor (%d) != hidden size (%d)" % (input_width , hidden_size))
    
    # cần giữ biểu diễn dưới dạng tensor 2D để tránh định hình lại nó từ tensor 3D thành tensor 2D
    # tạo một biến prev_output  = kết quả của phương thức reshape_to_maxtrix biến đổi 1 tensor 3d thành 2d
    prev_output = reshape_to_matrix(input_tensor)

    # tạo một danh sách để lưu chữ tất cả các lớp outputs 
    all_layer_outputs = []
    # duyệt qua 1 dnah scahs 0- > num_hidden_layers
    for layer_idx in range(num_hidden_layers):
        # tạo một biến variable_scope là một lớp theo chỉ số index và
        # với with ta đặt tất cả vào phạm vi của nó 
        with tf.variable_creator_scope("layer_%d", layer_idx):
            # gán biến layer_input = prev_output 
            layer_input = prev_output
            # tiếp tục tạo một biến phạm vi attention sử dụng with để xác định phạm vi chủa nó  
            with tf.variable_creator_scope("attention"):
                # tạo một danh scahs attention_head để lưu trữ danh sách đầu ra cho các lớp attention 
                attention_heads = []
                # tiếp tục tạo một biến phạm vi tự chú ý sử dụng with để xác định phạm vi của nó 
                with tf.variable_creator_scope("self"):
                    # khởi tạo một lớp head_attention gán nó bằng phương thức attention_layer 
                    attention_head = attention_layer(
                        from_tensor=layer_input,
                        to_tensor=layer_input,
                        attention_mask=attention_mask,
                        num_attention_heads=num_attention_heads,
                        size_per_head=attention_head_size,
                        attention_probs_dropout_prob=attention_probs_dropout_prob,
                        initializer_range=initializer_range,
                        do_return_2d_tensor=True,
                        batch_size=batch_size,
                        from_seq_length=seq_length,
                        to_seq_length=seq_length
                    )      
                    # sau đó thêm kết quả của lơp attention này vào dnah sách chứa kết quả của các lớp attention
                    attention_heads.append(attention_head)

                # tạo một biến attention_output sau đó ta gá đầu ra của lớp attention = None 
                attention_output = None 
                # kiểm tra xem số phần tử trong danh sách attention_heads có == 1 
                if len(attention_heads) == 1:
                    # gán giá trị đầu tiên trong dnah sách attention_heads cho biến attention_output 
                    attention_output = attention_heads[0]
                
                else: 
                    # trường hợp có dãy khác thì ta chỉ ghép vào đầu tự chú ý trước khi chiếu 
                    # tức là ta gán biến attention_output = kết quả của dnah sách attention_mask 
                    # theo chiều cuối cùng 
                    attention_output = tf.concat(attention_heads, axis=-1)

                # chạy phép chiếu tuyến tính của hidden_size sau khi thêm phần dư với lớp đầu ra 
                # tạo một biến phạm vi đầu ra sử dụng with để xác định phạm vi của nó 
                with tf.variable_creator_scope("output"):
                    # chiều tuyến tính kết quả đầu ra của khối attention 
                    attention_output = tf.keras.layers.Dense(
                        attention_output, hidden_size, 
                        kernel_initializer= create_initializer(initializer_range)
                    )
                    # thêm 1 lớp xử lý dropout 
                    attention_output = dropout(attention_output , hidden_dropout_prob)
                    # và cuỗi cùng là lớp bình thường hóa 
                    attention_output = layer_norm(attention_output + layer_input)

                # The activation is only applied to the "intermediate" hidden layer.
                # tạo một biến intermediate (ffn) là một biến phạm vi 
                # sử dụng with để xác định phạm vi của biến này 
            with tf.variable_scope("intermediate"):
                # gán biến intermediate_ output = kết quả của phéo chiếu tuyến tính 
                intermediate_output = tf.keras.layers.Dense(
                    attention_output,
                    # chiều biểu diễn 
                    intermediate_size,
                    # hàm kích hoạt 
                    activation=intermediate_act_fn,
                    # khởi tạo tham só 
                    kernel_initializer=create_initializer(initializer_range))

            # Down-project back to `hidden_size` then add the residual.
            # tương tự tạo mọt biến phạm vi đầu ra sử dụng with để quyết định phạm vị biến 
            with tf.variable_scope("output"):
                #gán cho biến layer_output là kết quả phéo chiếu tuyến tính cho kết qyar của lớp 
                # ffn ở trê
                layer_output = tf.keras.layers.Dense(
                    intermediate_output,
                    hidden_size,
                    kernel_initializer=create_initializer(initializer_range))
                # add 1 lớp dropout layers 
                layer_output = dropout(layer_output, hidden_dropout_prob)
                # + add layer_norm
                layer_output = layer_norm(layer_output + attention_output)
                # gán lại biến prev_output = layer_norm để vòng lặp thực hiện lại từ đầu 
                prev_output = layer_output
                # thêm kết quả này vào danh sách chưá các lớp đầu ra .
                all_layer_outputs.append(layer_output)

        # kiểm tra xem giá trị của do_return_all_layer có băng True: 
        if do_return_all_layers: 
            # tạo một dnah sách lưu trữ đầu ra cuối cùng 
            final_outputs = []
            # duyệt qua dnah cách lưu trữ các đầu ra của các lớp layer_attention trước đó 
            for layer_output in all_layer_outputs: 
                # định hình lại thành tensor có hình dạng ban đầu [batch_size, seq_length, hidden_size]
                final_output = reshape_from_matrix(layer_output, input_shape)
                # thêm nó vào danh sách final_outputs
                final_outputs.append(final_output)
                # trả về dnah sách đầu ra chứa các tensor với shape [batch_size, seq_length, hidden_size]
            return final_outputs
        else:
            # trườn hợp còn lại tức do_return tensor = False
            # thì ta phải lấy kết quả từ biến prev_output và thực hiện tương tự 
            final_output = reshape_from_matrix(prev_output, input_shape)
            # trả về dnah sách đầu ra chứa các tensor với shape [batch_size, seq_length, hidden_size]
            return final_output
        

# xây dựng phương thức trả về hình dạnh mong đợi cho 1 tensor 
# get_shape_list nhận đầu vào là 1 tensor , số chiều mong đợi , name 
def get_shape_list (tensor, expected_rank=None , name=None):
    """Returns a list of the shape of tensor, preferring static dimensions.

    Args:
        tensor: A tf.Tensor object to find the shape of.
        expected_rank: (optional) int. The expected rank of `tensor`. If this is
        specified and the `tensor` has a different rank, and exception will be
        thrown.
        name: Optional name of the tensor for the error message.

    Returns:
        A list of dimensions of the shape of tensor. All static dimensions will
        be returned as python integers, and dynamic dimensions will be returned
        as tf.Tensor scalars.
    """
    # kiểm tra xem có tồn tại tên của tensor không 
    if name is None: 
        # ta gán name = tensor.name 
        name = tensor.name 
    
    # sau đó kiểm tra xem số chiều mong muốn có = None 
    if expected_rank  is not None: 
        # nếu tồn tại giá trị này 
        # ta kiểm tra 3 giá trị đâu vào 
        assert_rank (tensor, expected_rank , name)
    
    # lấy ra hình dnagj của tensor 
    shape = tensor.shape.as_list()

    # tạo ra một dnah scahs none_static_indexes để lưu chỉ các chỉ số kích thước 
    non_static_indexes = []
    for (index, dim) in enumerate(shape):
        # kiểm tra xem dim là giá trị nghĩa là kichs thước chiều của tensor có = None
        if dim is None: 
            # thêm các chỉ số index vào dnah sách none_static 
            non_static_indexes.append(index)
            # nếu danh sách none_static không tồn tại 
    if not non_static_indexes:
        # trả về shape 
        return shape

    # lấy ra hình dnagj của tensor 
    dyn_shape = tf.shape(tensor)
    # duyệt qua danh sách các chỉ số của tensor 
    for index in non_static_indexes:
        # gán số kích thước chiều theo chỉ số của tensor shape bằng kích thước chiều 
        # theo chỉ số của tdyn_shape
        shape[index] = dyn_shape[index]
        # trả về kết quả 
    return shape


# xây dựng phương thức chuyển đổi hình dnagj tensor thành ma trận 
def reshape_to_matrix(input_tensor):
    """Reshapes a >= rank 2 tensor to a rank 2 tensor (i.e., a matrix)."""
    # lấy ra số chiều của tensor 
    ndims = input_tensor.shape.ndims
    # nếu tensor < 2 chiều 
    if ndims < 2:
        # đưa ra cảnh báo đầu vào phải có ít 2 chiều 
        raise ValueError("Input tensor must have at least rank 2. Shape = %s" %
                        (input_tensor.shape))
    # trường hợp có 2 chiều 
    if ndims == 2:
        # trả về kích thước mặt định của đầu vào 
        return input_tensor

    # lấy chiều cuối cùng gán vào biến width
    width = input_tensor.shape[-1]
    # định hình lại tensor input và bỏ đi 1 chiều với tham số  -1 nghĩa là nó sẽ tự suy ra dựa vào width
    output_tensor = tf.reshape(input_tensor, [-1, width])
    # trả về kết quả 
    return output_tensor


# xây dựng phuuwong thức chuyển 1 ka trận 2d thành 1 tensor 3D 
def reshape_from_matrix(output_tensor, orig_shape_list):
    """Reshapes a rank 2 tensor back to its original rank >= 2 tensor."""
    if len(orig_shape_list) == 2:
        return output_tensor

    # lấy ra các chiều của tensor output_tensor 
    output_shape = get_shape_list(output_tensor)

    orig_dims = orig_shape_list[0:-1] # lấy ra 2 chiều đầu tiên của tensor 
    # lấy ra kích thước theo chiều cuối cùng từ  output_shape
    width = output_shape[-1]
    # thêm chiều cuối cùng có shape = width để nó thành 1 tensor 3 chiều 
    return tf.reshape(output_tensor, orig_dims + [width])


# xây dựng phương thức để kiểm tra số chiều của tensor 
def assert_rank(tensor, expected_rank, name=None):
    """Raises an exception if the tensor rank is not of the expected rank.

    Args:
        tensor: A tf.Tensor to check the rank of.
        expected_rank: Python integer or list of integers, expected rank.
        name: Optional name of the tensor for the error message.

    Raises:
        ValueError: If the expected shape doesn't match the actual shape.
    """
    # kiểm tra name có tồn tại
    if name is None:
        # gán name tensor cho name
        name = tensor.name

    # tạo ra một từ điển để lưu trữ các giá trị Bool 
    expected_rank_dict = {}
    if isinstance(expected_rank, six.integer_types):
        expected_rank_dict[expected_rank] = True
    else:
        for x in expected_rank:
            expected_rank_dict[x] = True
    # lấy ra số chiều 
    actual_rank = tensor.shape.ndims
    # kiểm tra xem số chiều này có nằm trong từ điển không 
    # nếu không đưa ra cảnh báo 
    if actual_rank not in expected_rank_dict:
        scope_name = tf.get_variable_scope().name
        raise ValueError(
            "For the tensor `%s` in scope `%s`, the actual rank "
            "`%d` (shape = %s) is not equal to the expected rank `%s`" %
            (name, scope_name, actual_rank, str(tensor.shape), str(expected_rank)))

In [11]:
import numpy as np 
a = np.random.rand(1,5)
b = np.random.rand(5,1)
a *b  , a , b

(array([[0.28946578, 0.31157443, 0.27321728, 0.40430227, 0.31906998],
        [0.13888947, 0.14949749, 0.13109322, 0.19398952, 0.15309395],
        [0.13774609, 0.14826678, 0.13001402, 0.19239254, 0.15183363],
        [0.26689513, 0.28727989, 0.25191358, 0.37277742, 0.29419099],
        [0.32523142, 0.35007176, 0.30697529, 0.4542568 , 0.35849344]]),
 array([[0.48853087, 0.5258436 , 0.4611083 , 0.6823402 , 0.53849382]]),
 array([[0.59252302],
        [0.28430029],
        [0.28195984],
        [0.54632194],
        [0.66573361]]))

In [12]:
0.66573361 *  0.5258436 

0.35007175812339597